In [1]:
import pickle

movie = pickle.load( open( "movie_tweet_2.pkl", "rb" ) )

In [2]:
from copy import copy

movie_tweet = copy(movie)

In [3]:
import re

for idx, data in enumerate(movie_tweet):
    title = data['title'].lower()
    movie_conversation = []
    for tweet in data['tweet']:
        cleaned_tweet = re.sub('[^A-Za-z0-9_\s]', "", tweet.lower())
        tweet_list = cleaned_tweet.split(" ")
        if title in tweet_list:
            tweet_list.remove(title)
        if '' in tweet_list:
            space_count = tweet_list.count('')
            for _ in range(space_count):
                tweet_list.remove('')
    
        movie_conversation.append(' '.join(tweet_list))
    movie_tweet[idx]['tweet'] = movie_conversation

In [4]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

st = PorterStemmer()
stop = set(stopwords.words('english'))

tweet_token = copy(movie_tweet)

for idx, data in enumerate(movie_tweet):
    tweet_data = []
    for tweet in data['tweet']:
        token_and_stem = [st.stem(i) for i in word_tokenize(tweet)]
        
        filtered_words = [i for i in token_and_stem if i not in stop]
        tweet_data.append(filtered_words)        
    
    tweet_token[idx]['tweet'] = tweet_data

In [5]:
import pandas as pd

df = pd.DataFrame.from_dict(tweet_token)

In [6]:
tweet_words = []
for i in df.tweet:
    tweet = []
    for j in i:
        tweet.extend(j)
    tweet_words.append(tweet)

In [7]:
import gensim

Word2Vec_model = gensim.models.Word2Vec(sentences=tweet_words, min_count=5, )

/home/hyeonju/anaconda3/envs/ml_python/lib/python3.4/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [8]:
import numpy as np

tweet_Word2Vec = list(copy(df.tweet))

for idx, thread in enumerate(df.tweet):
    for t_idx, tweet in enumerate(thread):
        for w_idx, word in enumerate(tweet):
            try:
                tweet_Word2Vec[idx][t_idx][w_idx] = Word2Vec_model[word]
            except:
                tweet_Word2Vec[idx][t_idx][w_idx] = np.zeros(100)

In [9]:
max_word_size = 0
max_conversation_size = 0

for conversation_list in tweet_Word2Vec:
    if len(conversation_list) > max_conversation_size:
        max_conversation_size = len(conversation_list)
    for word_list in conversation_list:
        if len(word_list) > max_word_size:
            max_word_size = len(word_list)

In [10]:
max_word_size

27

In [11]:
max_conversation_size

9

In [12]:
for idx, thread in enumerate(tweet_Word2Vec):
    zeros_thread_size = max_conversation_size-len(thread)
    if zeros_thread_size < max_conversation_size:
            for _ in range(zeros_thread_size):
                tweet_Word2Vec[idx].append([np.zeros(100) for i in range(max_conversation_size)])
    for t_idx, tweet in enumerate(thread):
        zeros_word_size = max_word_size-len(tweet)
        if  zeros_word_size < max_word_size:
            for _ in range(zeros_word_size):
                tweet_Word2Vec[idx][t_idx].append(np.zeros(100))

In [13]:
tweet_Word2Vec= np.array(tweet_Word2Vec)

In [14]:
for idx, t_list in enumerate(tweet_Word2Vec):
    tweet_Word2Vec[idx] = np.array(tweet_Word2Vec[idx])
    for t_idx, w_list in enumerate(t_list):
        tweet_Word2Vec[idx][t_idx] = np.array(tweet_Word2Vec[idx][t_idx])

In [15]:
print(len(tweet_Word2Vec))
print(len(tweet_Word2Vec[0]))
print(len(tweet_Word2Vec[0][0]))
print(len(tweet_Word2Vec[0][0][0]))

7919
9
27
100


In [16]:
title = list(copy(df.title))
title_set = {movie:idx for idx, movie in enumerate(list(set(title)))}

title_index = [title_set[i] for i in title]

In [17]:
import tensorflow as tf

y_data = tf.one_hot(title_index, len(title_set), 
                    1.0, 0.0, -1)

In [18]:
y_data

<tf.Tensor 'one_hot:0' shape=(7919, 1847) dtype=float32>

In [19]:
X = tf.placeholder("float", shape=[None, 9, 27, 100])
# X = tf.placeholder("float", shape=[9, 27, 100])

Y = tf.placeholder("float", shape=[None, 1847])

In [20]:
def conv2d(x, W, b):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, s):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, s, s, 0], strides=[1, 1, 1, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases):
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv2 = conv2d(x, weights['wc2'], biases['bc2'])
    conv3 = conv2d(x, weights['wc3'], biases['bc3'])

    
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, 3)
    print(conv1)
    conv2 = maxpool2d(conv2, 3)
    print(conv2)
    conv3 = maxpool2d(conv3, 3)
    print(conv3)
    

    concat = tf.concat([conv1, conv2, conv3], 0)
    return concat


#     h_drop = tf.nn.dropout(self.h_pool_flat, 0.75)

In [21]:
# n_input = 784 # MNIST data input (img shape: 28*28)
# n_classes = 1847 # MNIST total classes (0-9 digits)
# dropout = 0.75 # Dropout, probability to keep units

weights = {
    'wc1': tf.Variable(tf.truncated_normal([2, 2, 100, 1])),
    'wc2': tf.Variable(tf.truncated_normal([3, 3, 100, 1])),
    'wc3': tf.Variable(tf.truncated_normal([4, 4, 100, 1])),
}

biases = {
    'bc1': tf.Variable(tf.random_normal([1])),
    'bc2': tf.Variable(tf.random_normal([1])),
    'bc3': tf.Variable(tf.random_normal([1])),
}

In [22]:
conv_net(X, weights, biases)

Tensor("MaxPool:0", shape=(?, 9, 27, 1), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 9, 27, 1), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 9, 27, 1), dtype=float32)


<tf.Tensor 'concat:0' shape=(?, 9, 27, 1) dtype=float32>

In [23]:
# RNN Model

rnn_input = conv_net(X, weights, biases)

Tensor("MaxPool_3:0", shape=(?, 9, 27, 1), dtype=float32)
Tensor("MaxPool_4:0", shape=(?, 9, 27, 1), dtype=float32)
Tensor("MaxPool_5:0", shape=(?, 9, 27, 1), dtype=float32)


In [24]:
tf.shape(Y)

<tf.Tensor 'Shape:0' shape=(2,) dtype=int32>

In [25]:
lstm_size = 1847 #output size

time_step_size = 9 #x input 갯수
batch_size = 1

cell = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=False)
# cell= tf.contrib.rnn.MultiRNNCell([cell]*10)

state = tf.zeros([1, cell.state_size])

rnn_input = tf.cast(rnn_input, tf.int32)

X_split = tf.split(0, time_step_size, rnn_input) #입력 array 모양 벡터의 array의 갯수

In [26]:
outputs, state = tf.nn.rnn(cell, X_split, state)

AttributeError: 'module' object has no attribute 'rnn'

In [ ]:
logits = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
targets = tf.reshape(Y, [-1])
weights = tf.ones(time_step_size*batch_size)

loss = tf.nn.seq2seq.sequence_loss_by_example(
        [logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.AdamOptimizer(0.01, 0.9).minimize(cost)